# Chapter 4 - Animations
 

## ValueTrackers

In [ ]:
from manim import *

In [ ]:
%%manim -v WARNING --renderer=opengl --disable_caching HelloManim
class HelloManim(Scene):
    def construct(self):
        self.camera.background_color = "#ece6e2"
        d = Dot()
        self.add(d)
        self.wait(10)
